In [1]:
import numpy as np
import pandas as pd
from scipy.cluster.hierarchy import linkage, dendrogram
import sys
sys.setrecursionlimit(10000)
import matplotlib.pyplot as plt
import matplotlib as mpl

In [2]:
genes = [item[0] for item in pd.read_csv(open('../data/myeloma/genes.csv'), header=None).values.tolist()]
upreg = [item[0] for item in pd.read_csv(open('../data/myeloma/upreg.csv'), header=None).values.tolist()]
downreg = [item[0] for item in pd.read_csv(open('../data/myeloma/downreg.csv'), header=None).values.tolist()]
stable = [item[0] for item in pd.read_csv(open('../data/myeloma/stable.csv'), header=None).values.tolist()]
tedown = [item[0] for item in pd.read_csv(open('../data/myeloma/tedown.csv'), header=None).values.tolist()]
teup = [item[0] for item in pd.read_csv(open('../data/myeloma/teup.csv'), header=None).values.tolist()]

indices = []
for gene in tedown:
    indices.append(genes.index(gene))
for gene in downreg:
    indices.append(genes.index(gene))
for gene in teup:
    indices.append(genes.index(gene))
for gene in stable:
    indices.append(genes.index(gene))
for gene in upreg:
    indices.append(genes.index(gene))

print len(downreg) + len(upreg) + len(stable) + len(tedown) + len(teup)

5687


In [ ]:
print len(downreg)
print len(upreg)
test = downreg
test.extend(upreg)
print len(set(test))

In [3]:
polya = np.log2(pd.read_csv(open('../data/myeloma/polya.csv'), sep=',', header=None).as_matrix()).T
ribosome = np.log2(pd.read_csv(open('../data/myeloma/ribosome.csv'), sep=',', header=None).as_matrix()).T
te = pd.read_csv(open('../data/myeloma/te.csv'), sep=',', header=0)
new_te = []
for gene in genes:
    new_te.append(te[te['GeneID'] == gene].values.tolist()[0][1:])
te = np.log2(np.asarray(new_te).T)

new_polya = (polya - np.reshape(polya[0,:], (1, 5680)))[1:,].T
new_ribosome = (ribosome - np.reshape(ribosome[0,:], (1, 5680)))[1:,].T
new_te = (te - np.reshape(te[0,:], (1, 5680)))[1:,].T
combined = np.concatenate((new_polya, new_ribosome, new_te), 1)
# combined = np.concatenate((new_polya, new_ribosome), 1)

combined_clusters = combined[indices,:]

In [ ]:
plt.clf()
C = np.loadtxt('../data/myeloma/color_map.txt')
heatmap = plt.pcolor(combined, vmin=-3, vmax=3, cmap=mpl.colors.ListedColormap(C/255.0)) # cmap=plt.cm.YlGnBu
plt.colorbar()
plt.show()

In [13]:
# combined = np.concatenate((new_polya, new_ribosome), 1)
clustering = linkage(combined, method='average', metric='correlation')
clustering.shape

(5679, 4)

In [14]:
result = dendrogram(clustering,labels=genes, no_plot=True)
new_order = result['ivl']
new_order.reverse()

In [ ]:
indices = []
for gene in new_order:
    indices.append(genes.index(gene))
combined = np.concatenate((new_polya, new_ribosome, new_te), 1)
combined_ordered = combined[indices,:]
plt.clf()
C = np.loadtxt('../data/myeloma/color_map.txt')
heatmap = plt.pcolor(combined_ordered, vmin=-3, vmax=3, cmap=mpl.colors.ListedColormap(C/255.0)) # cmap=plt.cm.YlGnBu
plt.colorbar()
plt.show()